# Alignment Notebook

In [2]:
import scanpy
import os
import argparse
import scanpy as sc
import numpy as np
from scvi.inference import UnsupervisedTrainer
from scvi.dataset import AnnDatasetFromAnnData
from scvi.models import VAE
import torch
import scanorama

# default scVI training parameters
use_cuda = True
retrain = False


/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [3]:
input_file_path = 'data/adata_small_test.h5ad'
output_file_path = 'data/adata_small_test_output.h5ad'

batch='batch'
tissue='all'
n_neighbors=25
n_pcs=20
cluster_resolution=5


### pick a batch correction method

# batch_correction = 'scVI'
batch_correction ='bbknn'
# batch_correction = 'scanorama'

In [ ]:
print("subsetting sapiens data")
sapiens = sc.read_h5ad("data/OnClass_data/data_used_for_training/tabula-muris-senis-facs_cell_ontology.h5ad")
# if tissue != 'all':
#     try:
#         sapiens = sapiens[sapiens.obs['tissue'] == tissue].copy()

print('reading data in')
adata = sc.read_h5ad(input_file_path)
print(adata)
# adata.X = adata.raw.X
adata = adata.concatenate(sapiens)
print(adata)

if batch_correction == 'bbknn':
    print('normalization & scaling')
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    adata = sc.pp.filter_genes_dispersion(adata, subset=False, min_disp=.5, max_disp=None,
                                          min_mean=.0125, max_mean=10, n_bins=20, n_top_genes=None,
                                          log=True, copy=True)
    sc.pp.log1p(adata)
    sc.pp.scale(adata, max_value=10, zero_center=False)
    # adata.uns['tissue_colors'] = list(tissue_color_dict.values())

    print('pca')
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pl.pca_overview(adata)

    print('neighs')
    # sc.pp.neighbors(adata)
    sc.external.pp.bbknn(adata,metric = 'euclidean',
                          batch_key=batch,
                          approx=True,
                          n_pcs=n_pcs, trim=None, n_trees=10,
                          use_faiss=True, set_op_mix_ratio=1.0, local_connectivity=1)

    print('umap computing')
    sc.tl.umap(adata, n_components=2)

    print('clustering')
    sc.tl.louvain(adata, resolution=cluster_resolution)
    sc.tl.leiden(adata, resolution=cluster_resolution)

    print('save h5ad and launch cellxgene')
    adata.write(output_file_path + "bbknn.h5ad")

if batch_correction == 'scVI':
    model_file = output_file_path + 'scVI.model.pkl'
    print('Define scVI Model')
    data = AnnDatasetFromAnnData(adata, batch_label='batch')
    vae = VAE(data.nb_genes, n_batch=data.n_batches,
              n_layers=3, n_latent=50, dispersion='gene-batch',
              reconstruction_loss='zinb')
    trainer = UnsupervisedTrainer(
        vae,
        data,
        train_size=0.99,
        use_cuda=use_cuda,
        frequency=5,
        data_loader_kwargs={"pin_memory": False},
        n_epochs_kl_warmup=10
    )
    print('train scVI')
    if retrain:
        trainer.train(n_epochs=1)
        torch.save(trainer.model.state_dict(), model_file)
    else:
        if os.path.isfile(model_file):
            trainer.model.load_state_dict(torch.load(model_file))
            trainer.model.eval()
        else:
            trainer.train(n_epochs=1)
            torch.save(trainer.model.state_dict(), model_file)

    posterior = trainer.create_posterior(
        trainer.model, data, indices=np.arange(len(data))
    ).sequential()
    latent = posterior.get_latent()
    adata.obsm['X_scvi'] = latent[0]
    print('umap computing')
    sc.pp.neighbors(adata, n_neighbors=100, n_pcs=30, use_rep="X_scvi")
    sc.tl.umap(adata)

    print('clustering')
    sc.tl.louvain(adata, resolution=cluster_resolution)
    sc.tl.leiden(adata, resolution=cluster_resolution)

    print('save h5ad and launch cellxgene')
    adata.write(output_file_path + "scVI.h5ad")

if batch_correction == 'scanorama':
    print('normalization & scaling & gene selection')
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch)
    var_select = adata.var.highly_variable_nbatches > 1
    var_genes = var_select.index[var_select]

    adatas = [adata[adata.obs[batch] == i, var_genes] for i in np.unique(adata.obs[batch])]
    integrated = scanorama.integrate_scanpy(adatas, dimred=50)
    integrated = np.concatenate(integrated)
    adata.obsm['X_scanorama'] = integrated
    import pdb
    pdb.set_trace()
    print('umap computing')
    sc.pp.neighbors(adata, n_neighbors=100, n_pcs=50, use_rep="X_scanorama")
    sc.tl.umap(adata)

    print('clustering')
    sc.tl.louvain(adata, resolution=cluster_resolution)
    sc.tl.leiden(adata, resolution=cluster_resolution)

    print('save h5ad and launch cellxgene')
    adata.write(output_file_path + "scanorama.h5ad")

   

subsetting sapiens data
